### Tools for debugging graphs in reflacx and in DGL

In [ ]:
from iou_graph import IOUGraph
from scanpath_graph import ScanPathGraph
from euclidean_graph import EuclideanGraph

from dgl_reflacx_tools.dgl_reflacx_collection import GraphCollection

In [ ]:
dgl_dataset_dir = 'datasets/reflacx_densnet225_iou'

In [ ]:
gdb = GraphCollection(dgl_dataset_dir, IOUGraph)

In [ ]:
pair = gdb.fetch_by_reflacx('1bdf3180-0209f001-967acab6-0b811ea2-3c2e13eb', 'P300R510107')

In [ ]:
r_ious = pair.reflacx_ious()

In [ ]:
d_ious = pair.dgl_ious()